#### To contribute to the question-answering, create a tool using the pre-screen form and use it in addition to the current tool (created from OnCore data) and use both with the agent to supplement QA

In [6]:
import os
import streamlit as st
import pandas as pd
from langchain.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI
from langchain.document_loaders import UnstructuredXMLLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain import PromptTemplate
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.callbacks import StreamingStdOutCallbackHandler

In [2]:
embeddings = OpenAIEmbeddings()
trials_search = FAISS.load_local("faiss_embeddings", embeddings)

In [3]:
loader = PyPDFLoader('./pre_screen.pdf')
pre_screen_docs = loader.load()

In [4]:
pre_screen_vectordb = FAISS.from_documents(pre_screen_docs, embeddings)

In [30]:
trials_tool = create_retriever_tool(
    trials_search.as_retriever(), 
    "search_clinical_trials_database",
    "Searches and returns documents regarding clinical trials"
)
pre_screen_tool = create_retriever_tool(
    pre_screen_vectordb.as_retriever(), 
    "template_to_search_for_trials",
    "Can be used to understand what questions patients ask"
)
tools = [trials_tool, pre_screen_tool]

In [31]:
llm = ChatOpenAI(temperature = 0, 
                 model_name="gpt-3.5-turbo",
                 streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
agent_executor = create_conversational_retrieval_agent(llm, tools )

In [32]:
res = agent_executor ({"input": "trials for blood cancer"})['output']

Here are some clinical trials related to blood cancer:

1. Title: A Study to Evaluate Long-Term Safety of CAR-T Cell Therapy in Patients with Hematologic Malignancies
   - Investigator: Susan O'Brien
   - Status: Open to accrual
   - Description: This is a non-interventional, long-term safety study of allogeneic CAR-T cell therapy in patients with hematologic malignancies. The purpose is to collect long-term observational data to identify and understand potential late side effects in patients who have received CAR-T cell therapies.
   - Phase: I
   - Treatment Type: Other
   - Disease Sites: Multiple Myeloma, Non-Hodgkin's Lymphoma, Hodgkin's Lymphoma, Lymphoid Leukemia, Myeloid and Monocytic Leukemia, Other Hematopoietic Leukemia
   - Sponsor: Caribou Biosciences, Inc.

2. Title: Blood Collection Protocol for the Analysis of Exosomes in Patients with Breast Cancer
   - Investigator: Ritesh Parajuli
   - Status: Open to accrual
   - Description: The purpose of this research study is to

In [33]:
res = agent_executor({"input": "eligibility criteria for trial 2"})['output']

The eligibility criteria for Trial 2, "Blood Collection Protocol for the Analysis of Exosomes in Patients with Breast Cancer," are as follows:

Inclusion Criteria:
- Female participants
- Over the age of 18
- Ability to understand and willingness to sign a written informed consent document
- Consent to be in the study, signed and dated an approved consent form, which conforms to federal and institutional guidelines
- Breast cancer patients with Stage I, Stage II, Stage III, or Stage IV
- Breast cancer patients with hormone Receptor +, Her-2 receptor positive, triple positive, or triple negative breast cancer

Exclusion Criteria:
- Physically or mentally incapable of giving verbal or written consent
- Other active cancers except for cured skin and in situ cervical cancer for cancer patients
- History of cancer diagnosis or prior exposure to chemotherapy agents for non-cancer participants

Please note that this is a summary of the eligibility criteria, and it is recommended to consult wi

In [34]:
res = agent_executor({"input": "what are some considerations to keep in mind when searching for a trial?"})['output']

When searching for a clinical trial, there are several considerations to keep in mind:

1. Eligibility Criteria: Each clinical trial has specific eligibility criteria that determine who can participate. Consider whether you meet the age, gender, health status, and other requirements specified in the trial.

2. Purpose and Objectives: Understand the purpose of the trial and its objectives. Determine if the trial is evaluating a new treatment, comparing treatments, or studying a specific aspect of a disease. Make sure the trial aligns with your needs and goals.

3. Phase of the Trial: Clinical trials are conducted in different phases, each serving a specific purpose. Phase 1 trials focus on safety and dosage, Phase 2 trials evaluate effectiveness, and Phase 3 trials compare the new treatment to standard treatments. Consider which phase is appropriate for your situation.

4. Location: Trials are conducted at specific medical centers or research institutions. Consider the location and whet

In [37]:
res = agent_executor({"input": "trials related to the nose"})['output']

I apologize, but I couldn't find specific clinical trial information related to nose conditions. The search results I obtained appear to be related to a pre-screening process for potential research volunteers and participants at the UCI Center for Clinical Research. It seems to be a questionnaire to determine eligibility for various studies.

To find clinical trials specifically related to nose conditions, I recommend using online clinical trial databases such as ClinicalTrials.gov or consulting with healthcare professionals who may have access to relevant trials and studies.

In [44]:
res = agent_executor({"input": "give me some trials for the throat"})['output']

Here are some clinical trials related to the throat:

1. Title: A Phase Ib, First-In-Human, Dose Escalation and Expansion, Multicenter Study of XMT-1660 in Participants with Solid Tumors
   - Description: A study of XMT-1660 in solid tumors.
   - Phase: Ib
   - Treatment Type: Treatment
   - Disease Sites: Lip, Oral Cavity and Pharynx, Colon, Pancreas, Larynx, Soft Tissue, Corpus Uteri, Stomach, Lung, Rectum, Anus, Other Digestive Organ, Ovary, Brain and Nervous System, Other Respiratory and Intrathoracic Organs, Other Urinary, Bones and Joints, Other Male Genital, Thyroid, Unknown Sites, Ill-Defined Sites, Melanoma, Skin, Liver, Other Skin, Kidney, Breast, Esophagus, Small Intestine, Cervix, Other Female Genital, Prostate, Urinary Bladder, Eye and Orbit, Other Endocrine System
   - Sponsor: Mersana Therapeutics, Inc.

2. Title: Tailor RT: A Randomized Trial of Regional Radiotherapy in Biomarker Low Risk Node Positive and T3N0 Breast Cancer
   - Description: A study comparing the effec

In [41]:
res = agent_executor({"input": "generate pre-screening questions for study 1 that takes the trial's eligibility criteria into account"})['output']

Here are the pre-screening questions for Study 1, taking into account the eligibility criteria:

1. Have you received any Mustang Bio CAR-T cell product in a Mustang Bio-sponsored clinical study?

2. Are you a Medicare beneficiary with Medicare as your primary insurance?

3. Do you meet the clinical criteria for Mild Cognitive Impairment (MCI) or Dementia as defined by the 2018 National Institute on Aging - Alzheimer's Association Research Framework?

4. Have you had a brain MRI and/or CT within the last 24 months?

5. Have you had a clinical laboratory assessment (complete blood count [CBC], standard blood chemistry profile, thyroid stimulating hormone [TSH], vitamin B12) within the last 12 months?

6. Are you able to tolerate amyloid PET required by the protocol, to be performed at a participating PET facility?

7. Are you English or Spanish speaking for the purposes of informed consent?

8. Are you willing and able to provide consent? Consent may be by proxy.

9. Can your neuropsych

In [65]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

vectorstore_info = VectorStoreInfo(
    name="clinical_trials",
    description="search the clinical trials DB and return relevant answers",
    vectorstore=trials_search
    
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(llm= llm, toolkit=toolkit, handle_parsing_errors=True)

In [67]:
agent_executor.run("give me the names of the trials for heart disease")

I should use the clinical_trials tool to search for trials related to heart disease.

OutputParserException: Could not parse LLM output: `I should use the clinical_trials tool to search for trials related to heart disease.`